In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
# Initialize SparkSession with Kafka package
spark = SparkSession.builder \
    .appName("KafkaStructuredStreamingExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.4") \
    .config("spark.jars", "/home/nisha/hadoop/spark/spark-3.3.4-bin-hadoop2/jars/postgresql-42.2.23.jar") \
    .getOrCreate()

24/06/06 11:13:51 WARN Utils: Your hostname, nisha-HP-Laptop-15s-dy3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.53 instead (on interface wlo1)
24/06/06 11:13:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nisha/hadoop/spark1/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nisha/.ivy2/cache
The jars for the packages stored in: /home/nisha/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6eb26011-b659-44ee-aff3-c5044568b4f9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.4 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolu

In [3]:
# # Create Spark session with JDBC driver
# spark = SparkSession.builder \
#     .appName("KafkaToPostgres") \
#     .config("spark.jars",'/home/nisha/hadoop/spark/spark-3.3.4-bin-hadoop2/jars/postgresql-42.2.23.jar') \
#     .getOrCreate()

In [4]:
# Define the schema for the JSON data
schema = StructType([
    StructField("name", StringType(), True),
    StructField("id", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("place", StringType(), True),
    StructField("country", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone number", StringType(), True)
])

In [5]:

# Create DataFrame representing the stream of input lines from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "mytopic") \
    .option("startingOffsets", "earliest") \
    .load()

raw_df = df.selectExpr("CAST(value AS STRING)")


In [6]:
# Parse the JSON data and apply the schema
value_df = raw_df.select(from_json(col("value"), schema).alias("data"))

ex_df = value_df.selectExpr(
    "data.name as Name",
    "data.id as Id",
    "data.age as Age",
    "data.gender as Gender",
    "data.place as Place",
    "data.country as Country",
    "data.email as Email",
    "data.`phone number` as Phone"
)

In [7]:
# Define PostgreSQL connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/kafka_data"
jdbc_properties = {
    "user": "nisha",
    "password": "123",
    "driver": "org.postgresql.Driver"
}


In [8]:
# Function to write to PostgreSQL
def write_to_postgres(df, epoch_id):
    try:
        df.write.jdbc(url=jdbc_url, table="kafka_data1", mode="append", properties=jdbc_properties)
    except Exception as e:
        print(f"Error writing to PostgreSQL: {e}")

In [ ]:
# Write the stream to PostgreSQL
query = ex_df.writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("append") \
    .start()

query.awaitTermination()

24/06/06 11:14:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-980ccc35-85ca-4859-949d-46e108a3fea6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/06 11:14:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/06 11:14:07 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/06/06 11:14:07 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/06/06 11:14:07 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/06/06 11:14:07 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con